In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.4 MB/s eta 0:00:00


In [ ]:
import os

os.kill(os.getpid(), 9)

In [ ]:
!python -m pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ev5txag1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ev5txag1
  Resolved https://github.com/huggingface/transformers to commit 717dadc6f36be9f50abc66adfd918f9b0e6e3502
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.31.0.dev0-py3-none-any.whl size=7347790 sha256=1fbb7c4a749360b2f0410b509a5df52bbeeb13952c16164895001799c6c56e12
  Stored in directory: /tmp/pip-ephem-wheel-cache-bv97m9xp/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.30.2
    Uninstalling transformers-4.30.2:
      Successfully uninstalled transformers-4.30.2


In [ ]:
import pandas as pd
import csv
from collections import defaultdict
import matplotlib.pyplot as plt
import os
import ast
import numpy as np
import torch
from torch.utils.data import Dataset, random_split
from transformers import GPT2Tokenizer, TrainingArguments, Trainer, GPT2LMHeadModel
import datetime


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['TORCH_USE_CUDA_DSA'] = "1"

In [ ]:
PATH = "/content/gdrive/MyDrive/NLP/github/NLP_Project/"

In [ ]:
def log(message, task):
    if task is None:
        return
    if (not os.path.exists(task)):
        os.makedirs(f'{PATH}logs', exist_ok=True)
        with open(f'{PATH}logs/{task}', 'w') as f:
            f.write(f"[{datetime.datetime.now()}] {message.encode('utf-8')}\n")
    else:
        with open(f'{PATH}logs/{task}', 'a') as f:
            f.write(f"[{datetime.datetime.now()}] {message.encode('utf-8')} \n")


**Code copied from this medium post - https://medium.com/geekculture/fine-tune-eleutherai-gpt-neo-to-generate-netflix-movie-descriptions-in-only-47-lines-of-code-40c9b4c32475**

In [ ]:
class Dataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            encodings_dict = tokenizer('<|startoftext|>' + txt + '<|endoftext|>', truncation=True,
                                       max_length=max_length, padding="max_length")
            self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
            self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

In [ ]:
torch.manual_seed(42)

### Loading GPT2-Medium Model from 🤗 Model Hub

In [ ]:
def get_model():
  tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium', bos_token='<|startoftext|>',eos_token='<|endoftext|>', pad_token='<|pad|>')
  model = GPT2LMHeadModel.from_pretrained('gpt2-medium').to(device)
  model.resize_token_embeddings(len(tokenizer))
  return model,tokenizer


### GPT Generated Description

In [ ]:
def generate_description(tokenizer,model,category):
  generated = tokenizer("<|startoftext|> ", return_tensors="pt").input_ids.cuda()
  sample_outputs = model.generate(generated, do_sample=True, top_k=50,
                                  max_length=300, top_p=0.95, temperature=1.9, num_return_sequences=20)
  for i, sample_output in enumerate(sample_outputs):
      print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))
      log("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)),f"language_model: {category}")
      # pd.options.display.max_colwidth = 1000
      # descriptions.sample(10)

In [ ]:
def get_dataset(category,tokenizer):

  df = pd.read_csv(f'{PATH}/data/traindata/{category}.csv')
  data = df.to_dict('dict')

  descriptions = list(data['about'].values())
  max_length = max([len(tokenizer.encode(description)) for description in descriptions])

  dataset = Dataset(descriptions, tokenizer, max_length=max_length)
  train_size = int(0.9 * len(dataset))
  train_dataset, val_dataset = random_split(dataset, [train_size, len(dataset) - train_size])

  return train_dataset, val_dataset


In [ ]:
training_args = TrainingArguments(output_dir=f'{PATH}models/', num_train_epochs=1, logging_steps=100, save_steps=5000,
                                  per_device_train_batch_size=1, per_device_eval_batch_size=1,
                                  warmup_steps=10, weight_decay=0.05, logging_dir=f'{PATH}logs', report_to = 'none')

In [ ]:
categories = ['action','adventure','rpg','strategy','simulation','sports_and_racing']
for category  in categories:
  print(category)
  model,tokenizer = get_model()
  train_dataset, val_dataset = get_dataset(category,tokenizer)
  torch.cuda.empty_cache()
  Trainer(model=model,  args=training_args, train_dataset=train_dataset,
          eval_dataset=val_dataset, data_collator=lambda data: {'input_ids': torch.stack([f[0] for f in data]),
                                                                  'attention_mask': torch.stack([f[1] for f in data]),
                                                                  'labels': torch.stack([f[0] for f in data])}).train()
  generate_description(tokenizer,model,category)
  torch.save(model.state_dict(), f'{PATH}models/{category}.language_model.pt')


action


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,4.069100
200,1.200900
300,1.207500
400,1.138500


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  
There Are a Good Half to all to all You to find
  
A game made by the team at Frogstud (Switzerland) who creates games like Space Knight's Return: Knights, Space Marines, War of Magic
is currently being produced by Team 811 Team (Switzerland).
You will need: 

6 Hours
5,24 Dollars
 6,88 Dollars,1,88.66  Dollars
3 Cars 
4,16.4,5 Cars 
5
-P maps (3 levels available on Steam)
Ships 1/7 Cars & 6 Cars 
Corsican Racetrack in which the player have to beat an opponent while running into and dodging obstacles in this awesome arcade game experience!
To complete:
5-Minute-long game modes or Fast gameplay (Fast paced battles)
-Unobtrusive elements in your game experience; no hard to the right thing is hidden; just good music.
Team88 has worked to recreate both the aesthetic of its favorite Super Brothers 3 and Ninja Team Heroes, in order, in that direction. Super Sisters  Team heroes was designed with all-enemies to keep up on a quest; to protect our favorite players like myself is most defin

...

sports_and_racing


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,6.123800
200,2.065000
300,1.991800
400,1.903900


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0:  Rally is an arcade driving adventure style ride based in The Game Park. 
The Challenge features 4 stages and 20 races each themed with one theme of 'Car Free,' 'Fun' - each stage you're gonna want to race with some variety as the difficulty of 'The Challenge' affects your win rate so be sure to race long enough in your bike and drive yourself insane for more. Make a difference, and prove that your car-pilot's skills and speed record is higher if you finish with 
 

1:  
Race in a track and obstacle challenge game, try you will to win an amazing amount of medals every race.
 Play without mouse while running. Game controls, control pad etc. in a completely smooth racing experience! Use mouse/A button to break away and score points fast while trying not to land a wrong set. Earn coins, upgrade equipment (to keep competitive?), collect gold medals but avoid hazards more by aiming than jumping! There are other game mode of choice and various racing locations worldwide which can only mak

  torch.save(model.state_dict(), f'{PATH}models/{category}.language_model.pt')
